# 2023 Project

You have to work on the [ZTBus: A Large Dataset of Time-Resolved City Bus Driving Missions](https://www.research-collection.ethz.ch/handle/20.500.11850/626723) repository.

It contains:
*  [metaData.csv](https://www.research-collection.ethz.ch/bitstream/handle/20.500.11850/626723/metaData.csv?sequence=1&isAllowed=y), shortly *trips*
*  several other files containing detailed data on some bus parameters, whose name is in the *trips* file. Those files can be downloaded as a [zip file](https://www.research-collection.ethz.ch/bitstream/handle/20.500.11850/626723/ZTBus_compressed.zip?sequence=3&isAllowed=y). Let us call those datasets the *details* datasets.

### Notes

1.    It is mandatory to use GitHub for developing the project.
1.    The project must be a jupyter notebook.
1.    There is no restriction on the libraries that can be used, nor on the Python version.
1.    All questions on the project **must** be asked in a public channel on [Zulip](https://focs.zulipchat.com).
1.    At most 3 students can be in each group. You must create the groups by yourself. You can use the Zulip channel to create the groups.
1.    You do not have to send me the project *before* the discussion.

In [1]:
import pandas as pd
import re
import zipfile
import numpy as np
import datetime

### 1. Extract all trips with `busRoute` 83

In [2]:
trips = pd.read_csv("https://www.research-collection.ethz.ch/bitstream/handle/20.500.11850/626723/metaData.csv?sequence=1&isAllowed=y", na_values="-", dtype={"busRoute":"string"})

In [3]:
trips.head()

,name,busNumber,startTime_iso,startTime_unix,endTime_iso,endTime_unix,drivenDistance,busRoute,energyConsumption,itcs_numberOfPassengers_mean,itcs_numberOfPassengers_min,itcs_numberOfPassengers_max,status_gridIsAvailable_mean,temperature_ambient_mean,temperature_ambient_min,temperature_ambient_max
0,B183_2019-04-30_03-18-56_2019-04-30_08-44-20,183,2019-04-30T03:18:56Z,1556594336,2019-04-30T08:44:20Z,1556613860,77213.87,<NA>,4.785852e+08,5.538860,0,20,0.740640,282.3780,281.15,293.15
1,B183_2019-04-30_13-22-07_2019-04-30_17-54-02,183,2019-04-30T13:22:07Z,1556630527,2019-04-30T17:54:02Z,1556646842,59029.60,31,4.022585e+08,33.114580,4,74,0.855234,287.5443,285.15,293.15
2,B183_2019-05-01_05-58-51_2019-05-01_22-32-30,183,2019-05-01T05:58:51Z,1556690331,2019-05-01T22:32:30Z,1556749950,240900.40,33,1.445733e+09,19.689140,0,55,0.777860,288.7490,280.15,294.15
3,B183_2019-05-03_02-50-21_2019-05-03_05-53-20,183,2019-05-03T02:50:21Z,1556851821,2019-05-03T05:53:20Z,1556862800,42565.48,<NA>,2.819867e+08,1.685185,0,8,0.767122,282.4129,281.15,292.15
4,B183_2019-05-03_15-41-57_2019-05-03_23-06-24,183,2019-05-03T15:41:57Z,1556898117,2019-05-03T23:06:24Z,1556924784,125277.20,72,6.207258e+08,23.753570,1,67,0.907342,284.7325,282.15,287.15


In [4]:
trips[trips["busRoute"] == "83"]

,name,busNumber,startTime_iso,startTime_unix,endTime_iso,endTime_unix,drivenDistance,busRoute,energyConsumption,itcs_numberOfPassengers_mean,itcs_numberOfPassengers_min,itcs_numberOfPassengers_max,status_gridIsAvailable_mean,temperature_ambient_mean,temperature_ambient_min,temperature_ambient_max
154,B183_2020-03-03_04-42-38_2020-03-03_19-44-51,183,2020-03-03T04:42:38Z,1583210558,2020-03-03T19:44:51Z,1583264691,225047.90,83,1.544278e+09,23.47531,0,118,0.472180,280.5450,279.15,289.1500
155,B183_2020-03-06_04-53-23_2020-03-06_19-44-42,183,2020-03-06T04:53:23Z,1583470403,2020-03-06T19:44:42Z,1583523882,224512.30,83,1.631816e+09,17.41578,0,69,0.451028,279.8850,278.15,289.1500
157,B183_2020-03-09_14-16-13_2020-03-09_19-34-17,183,2020-03-09T14:16:13Z,1583763373,2020-03-09T19:34:17Z,1583782457,77824.36,83,5.406013e+08,23.18182,0,74,0.460099,281.0489,279.15,291.1500
158,B183_2020-03-10_04-50-03_2020-03-10_19-51-25,183,2020-03-10T04:50:03Z,1583815803,2020-03-10T19:51:25Z,1583869885,225095.80,83,1.692171e+09,20.96410,0,86,0.475233,279.8363,279.15,291.1500
159,B183_2020-03-12_04-56-41_2020-03-12_19-44-57,183,2020-03-12T04:56:41Z,1583989001,2020-03-12T19:44:57Z,1584042297,224181.20,83,1.145860e+09,17.21235,0,80,0.340882,287.3445,282.15,291.1500
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1399,B208_2022-11-30_04-47-53_2022-11-30_19-50-22,208,2022-11-30T04:47:53Z,1669783673,2022-11-30T19:50:22Z,1669837822,223165.00,83,1.560888e+09,27.89066,2,100,0.456196,280.6948,279.15,293.1500
1400,B208_2022-12-01_05-19-41_2022-12-01_18-20-57,208,2022-12-01T05:19:41Z,1669871981,2022-12-01T18:20:57Z,1669918857,190196.00,83,1.418847e+09,26.03927,0,96,0.450413,279.7655,279.15,292.1500
1401,B208_2022-12-02_04-47-48_2022-12-02_19-40-01,208,2022-12-02T04:47:48Z,1669956468,2022-12-02T19:40:01Z,1670010001,224473.40,83,1.611150e+09,24.80384,2,91,0.438693,279.7888,279.15,291.1500
1405,B208_2022-12-07_05-13-02_2022-12-07_19-19-53,208,2022-12-07T05:13:02Z,1670389982,2022-12-07T19:19:53Z,1670440793,210041.60,83,1.536697e+09,28.78539,0,115,0.434858,279.5283,278.15,292.6655


### 2. Extract all trips where `busRoute` is not a number

In [5]:
trips[trips["busRoute"].str.contains(r"\D+")]

,name,busNumber,startTime_iso,startTime_unix,endTime_iso,endTime_unix,drivenDistance,busRoute,energyConsumption,itcs_numberOfPassengers_mean,itcs_numberOfPassengers_min,itcs_numberOfPassengers_max,status_gridIsAvailable_mean,temperature_ambient_mean,temperature_ambient_min,temperature_ambient_max
533,B183_2021-12-18_23-37-00_2021-12-19_03-38-35,183,2021-12-18T23:37:00Z,1639870620,2021-12-19T03:38:35Z,1639885115,76216.06,N4,481350300.0,9.198582,0,37,0.491653,276.8632,275.15,288.15
553,B183_2022-01-07_23-40-43_2022-01-08_03-31-21,183,2022-01-07T23:40:43Z,1641598843,2022-01-08T03:31:21Z,1641612681,68557.06,N2,453625100.0,4.626984,0,13,0.427488,276.9673,275.15,287.15
554,B183_2022-01-08_23-40-17_2022-01-09_03-35-32,183,2022-01-08T23:40:17Z,1641685217,2022-01-09T03:35:32Z,1641699332,67962.92,N2,475383300.0,7.495495,0,26,0.515514,278.5645,277.15,288.15
561,B183_2022-01-15_23-41-46_2022-01-16_03-40-23,183,2022-01-15T23:41:46Z,1642290106,2022-01-16T03:40:23Z,1642304423,77156.70,N1,525168300.0,6.512500,0,32,0.473809,274.9937,273.15,286.15
568,B183_2022-01-21_23-35-40_2022-01-22_03-26-24,183,2022-01-21T23:35:40Z,1642808140,2022-01-22T03:26:24Z,1642821984,71917.75,N2,455476000.0,5.357143,0,23,0.493608,275.3073,274.15,281.15
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1373,B208_2022-10-21_22-38-32_2022-10-22_02-42-21,208,2022-10-21T22:38:32Z,1666391912,2022-10-22T02:42:21Z,1666406541,78567.16,N1,434776600.0,16.333330,0,45,0.431852,289.2550,288.15,296.15
1374,B208_2022-10-22_22-34-45_2022-10-23_02-29-59,208,2022-10-22T22:34:45Z,1666478085,2022-10-23T02:29:59Z,1666492199,73427.97,N2,399773700.0,17.710530,0,57,0.443358,287.3486,285.15,295.15
1394,B208_2022-11-25_23-35-16_2022-11-26_03-30-39,208,2022-11-25T23:35:16Z,1669419316,2022-11-26T03:30:39Z,1669433439,72911.26,N2,447553400.0,11.216670,1,32,0.465024,281.3884,280.15,293.15
1407,B208_2022-12-09_23-55-12_2022-12-10_03-24-28,208,2022-12-09T23:55:12Z,1670630112,2022-12-10T03:24:28Z,1670642668,59548.57,N1,451916500.0,20.105260,0,74,0.495739,279.4540,277.15,291.15


### 3. For each (busNumber, busRoute) pair, determine the number of trips

In [6]:
trips.groupby(["busNumber", "busRoute"])["name"].count()

busNumber  busRoute
183        31           12
           32           12
           33          130
           46          104
           72          114
           83          441
           N1           10
           N2           19
           N4           11
208        31            5
           32           14
           33           25
           46           19
           72           44
           83          405
           N1            6
           N2           20
           N4            7
Name: name, dtype: int64

### 4. For each trip, compute the ratio between the energy consumption and the average number of passengers

In [7]:
trips["energy_passengers_ratio"] = trips["energyConsumption"] / trips["itcs_numberOfPassengers_mean"]
trips[["energyConsumption", "itcs_numberOfPassengers_mean", "energy_passengers_ratio"]]

,energyConsumption,itcs_numberOfPassengers_mean,energy_passengers_ratio
0,4.785852e+08,5.538860,8.640500e+07
1,4.022585e+08,33.114580,1.214747e+07
2,1.445733e+09,19.689140,7.342794e+07
3,2.819867e+08,1.685185,1.673328e+08
4,6.207258e+08,23.753570,2.613190e+07
...,...,...,...
1404,4.260419e+08,39.808990,1.070215e+07
1405,1.536697e+09,28.785390,5.338462e+07
1406,1.415700e+09,29.879400,4.738047e+07
1407,4.519165e+08,20.105260,2.247753e+07


### 5. For each station (`itcs_stopName`), determine the average number of passengers.

In [8]:
archive = zipfile.ZipFile("ZTBus_compressed.zip", mode= "r")

In [9]:
list_of_trips=[]
for name in archive.namelist():
    if name.endswith('.csv'):
        list_of_trips.append(pd.read_csv(archive.open(name), usecols=["time_unix", "odometry_vehicleSpeed","itcs_numberOfPassengers", "itcs_stopName", "gnss_altitude", "status_haltBrakeIsActive", "status_parkBrakeIsActive"], na_values="-", dtype = {"status_haltBrakeIsActive":"Int64", "status_parkBrakeIsActive":"Int64", "gnss_altitude":"float"}, low_memory=False))
        list_of_trips[-1].sort_values(by = ["time_unix"], inplace = True)
        list_of_trips[-1]["Mission"]=name

In [10]:
details = pd.concat(list_of_trips, ignore_index=True)

In [11]:
details.head()

,time_unix,gnss_altitude,itcs_numberOfPassengers,itcs_stopName,odometry_vehicleSpeed,status_haltBrakeIsActive,status_parkBrakeIsActive,Mission
0,1556594336,NaN,NaN,NaN,0.0,0,0,B183_2019-04-30_03-18-56_2019-04-30_08-44-20.csv
1,1556594337,NaN,NaN,NaN,0.0,0,0,B183_2019-04-30_03-18-56_2019-04-30_08-44-20.csv
2,1556594338,NaN,NaN,NaN,0.0,0,0,B183_2019-04-30_03-18-56_2019-04-30_08-44-20.csv
3,1556594339,NaN,NaN,NaN,0.0,0,0,B183_2019-04-30_03-18-56_2019-04-30_08-44-20.csv
4,1556594340,NaN,NaN,NaN,0.0,0,0,B183_2019-04-30_03-18-56_2019-04-30_08-44-20.csv


In [12]:
details.groupby("itcs_stopName", as_index=False)["itcs_numberOfPassengers"].mean().round(3)

,itcs_stopName,itcs_numberOfPassengers
0,"Zürich, Albisrank",16.197
1,"Zürich, Albisriederplatz",25.453
2,"Zürich, Altes Krematorium",21.980
3,"Zürich, Bahnhof Affoltern",2.851
4,"Zürich, Bahnhof Altstetten",7.817
...,...,...
144,"Zürich, Zentrum Witikon",17.255
145,"Zürich, Zweiackerstrasse",7.502
146,"Zürich, Zwielplatz",2.049
147,"Zürich, Zwinglihaus",23.197


### 6. For each station, determine the buses that have stopped there at least once.

In [13]:
details["bus_number"] = details["Mission"].str.extract(r"\w(\d+)")

In [14]:
stop_count = details.groupby(["itcs_stopName","bus_number"])["bus_number"].count()
stop_count

itcs_stopName                   bus_number
Zürich, Albisrank               183            6324
                                208            6199
Zürich, Albisriederplatz        183           11218
                                208            7633
Zürich, Altes Krematorium       183           10022
                                              ...  
Zürich, Zwielplatz              208               4
Zürich, Zwinglihaus             183            2791
                                208            1231
Zürich,Kalkbreite/Bhf.Wiedikon  183             279
                                208             256
Name: bus_number, Length: 297, dtype: int64

### 7. For each station, determine the buses that have stopped there at least ten times.

In [15]:
stop_count[stop_count > 9]

itcs_stopName                   bus_number
Zürich, Albisrank               183            6324
                                208            6199
Zürich, Albisriederplatz        183           11218
                                208            7633
Zürich, Altes Krematorium       183           10022
                                              ...  
Zürich, Zwielplatz              183              77
Zürich, Zwinglihaus             183            2791
                                208            1231
Zürich,Kalkbreite/Bhf.Wiedikon  183             279
                                208             256
Name: bus_number, Length: 295, dtype: int64

### 9. For each (route, bus) pair, compute the ratio between the overall energy consumption and the overall driven distance. 

In [16]:
overall_values= trips.groupby(["busRoute", "busNumber"], as_index=False)[[ "energyConsumption", "drivenDistance"]].sum()
overall_values.head()

,busRoute,busNumber,energyConsumption,drivenDistance
0,31,183,1.387782e+10,2279952.00
1,31,208,5.535865e+09,1046306.68
2,32,183,1.393320e+10,2256540.10
3,32,208,1.283340e+10,2337064.21
4,33,183,1.761996e+11,29510481.41


In [17]:
overall_values["energy_distance_ratio"] = round(overall_values["energyConsumption"] / overall_values["drivenDistance"], 3)
overall_values.head()

,busRoute,busNumber,energyConsumption,drivenDistance,energy_distance_ratio
0,31,183,1.387782e+10,2279952.00,6086.892
1,31,208,5.535865e+09,1046306.68,5290.863
2,32,183,1.393320e+10,2256540.10,6174.585
3,32,208,1.283340e+10,2337064.21,5491.248
4,33,183,1.761996e+11,29510481.41,5970.748


### 10. Starting from the results of the previous point, for each route compute the buses with max and min energy ratio, and save the difference between these ratios in a dataframe.

In [18]:
max_ratio = overall_values.iloc[overall_values.groupby("busRoute")["energy_distance_ratio"].idxmax()][["busRoute", "busNumber", "energy_distance_ratio"]]
max_ratio

,busRoute,busNumber,energy_distance_ratio
0,31,183,6086.892
2,32,183,6174.585
4,33,183,5970.748
6,46,183,5631.036
8,72,183,5899.030
10,83,183,5844.977
12,N1,183,5980.921
14,N2,183,5701.850
16,N4,183,6154.987


In [19]:
min_ratio = overall_values.iloc[overall_values.groupby("busRoute")["energy_distance_ratio"].idxmin()][["busRoute", "busNumber", "energy_distance_ratio"]]
min_ratio

,busRoute,busNumber,energy_distance_ratio
1,31,208,5290.863
3,32,208,5491.248
5,33,208,5639.245
7,46,208,5583.111
9,72,208,5410.319
11,83,208,5819.639
13,N1,208,5640.062
15,N2,208,5405.153
17,N4,208,6067.190


In [20]:
difference = pd.merge(max_ratio, min_ratio, on=["busRoute"], suffixes=["_max", "_min"])
difference

,busRoute,busNumber_max,energy_distance_ratio_max,busNumber_min,energy_distance_ratio_min
0,31,183,6086.892,208,5290.863
1,32,183,6174.585,208,5491.248
2,33,183,5970.748,208,5639.245
3,46,183,5631.036,208,5583.111
4,72,183,5899.030,208,5410.319
5,83,183,5844.977,208,5819.639
6,N1,183,5980.921,208,5640.062
7,N2,183,5701.850,208,5405.153
8,N4,183,6154.987,208,6067.190


In [21]:
difference["difference_ratio"] = difference["energy_distance_ratio_max"] - difference["energy_distance_ratio_min"]
difference

,busRoute,busNumber_max,energy_distance_ratio_max,busNumber_min,energy_distance_ratio_min,difference_ratio
0,31,183,6086.892,208,5290.863,796.029
1,32,183,6174.585,208,5491.248,683.337
2,33,183,5970.748,208,5639.245,331.503
3,46,183,5631.036,208,5583.111,47.925
4,72,183,5899.030,208,5410.319,488.711
5,83,183,5844.977,208,5819.639,25.338
6,N1,183,5980.921,208,5640.062,340.859
7,N2,183,5701.850,208,5405.153,296.697
8,N4,183,6154.987,208,6067.190,87.797


### 11. Find the bus maximizing the difference computed in the previous point.

In [22]:
difference.iloc[difference["difference_ratio"].idxmax()][["busNumber_max", "busNumber_min"]]

busNumber_max    183
busNumber_min    208
Name: 0, dtype: object

### 12. Extract the rows of the details such that the `gnss_altitude` differs from the value in the preceding row. Store also the difference in the variable `altitude_variation`.

In [23]:
details["altitude_variation"] = details["gnss_altitude"] - details["gnss_altitude"].shift(periods=-1)

In [24]:
details[(details["altitude_variation"]!= 0) & ~(details["altitude_variation"].isnull())]

,time_unix,gnss_altitude,itcs_numberOfPassengers,itcs_stopName,odometry_vehicleSpeed,status_haltBrakeIsActive,status_parkBrakeIsActive,Mission,bus_number,altitude_variation
147,1556594483,397.8000,NaN,NaN,3.544101,0,0,B183_2019-04-30_03-18-56_2019-04-30_08-44-20.csv,183,-0.0299
148,1556594484,397.8299,NaN,NaN,3.793694,0,0,B183_2019-04-30_03-18-56_2019-04-30_08-44-20.csv,183,-2.4841
149,1556594485,400.3140,NaN,NaN,4.045902,0,0,B183_2019-04-30_03-18-56_2019-04-30_08-44-20.csv,183,-1.3752
150,1556594486,401.6892,NaN,NaN,4.404461,0,0,B183_2019-04-30_03-18-56_2019-04-30_08-44-20.csv,183,1.2032
151,1556594487,400.4860,NaN,NaN,4.860109,0,0,B183_2019-04-30_03-18-56_2019-04-30_08-44-20.csv,183,1.3668
...,...,...,...,...,...,...,...,...,...,...
48674430,1670729314,416.8195,NaN,NaN,0.000000,1,1,B208_2022-12-10_23-34-46_2022-12-11_03-29-05.csv,208,-0.1718
48674431,1670729315,416.9913,NaN,NaN,0.000000,1,1,B208_2022-12-10_23-34-46_2022-12-11_03-29-05.csv,208,-0.1999
48674432,1670729316,417.1912,NaN,NaN,0.000000,1,1,B208_2022-12-10_23-34-46_2022-12-11_03-29-05.csv,208,1.1372
48674456,1670729340,410.1028,NaN,NaN,0.000000,0,1,B208_2022-12-10_23-34-46_2022-12-11_03-29-05.csv,208,-0.0888


### 13. For each details dataset, compute the sum of the absolute value (i.e. the sign is not considered) of `altitude_variation`.

In [25]:
details["abs_altitude_variation"]= details["altitude_variation"].abs()

In [26]:
details.groupby("Mission")["abs_altitude_variation"].sum()

Mission
B183_2019-04-30_03-18-56_2019-04-30_08-44-20.csv      184.7389
B183_2019-04-30_13-22-07_2019-04-30_17-54-02.csv     3896.8300
B183_2019-05-01_05-58-51_2019-05-01_22-32-30.csv      637.4071
B183_2019-05-03_02-50-21_2019-05-03_05-53-20.csv     4074.4802
B183_2019-05-03_15-41-57_2019-05-03_23-06-24.csv    13364.1739
                                                       ...    
B208_2022-12-06_14-43-49_2022-12-06_18-22-52.csv     2300.3956
B208_2022-12-07_05-13-02_2022-12-07_19-19-53.csv     8617.6752
B208_2022-12-08_05-22-20_2022-12-08_18-39-15.csv     8022.7330
B208_2022-12-09_23-55-12_2022-12-10_03-24-28.csv     2748.4226
B208_2022-12-10_23-34-46_2022-12-11_03-29-05.csv     3247.2228
Name: abs_altitude_variation, Length: 1409, dtype: float64

### 14.  For each month of the year, compute the average ambient temperature

In [27]:
trips["start_datetime"] = pd.to_datetime(trips["startTime_iso"], utc=True)

In [28]:
trips.groupby([trips.start_datetime.dt.year, trips.start_datetime.dt.month])["temperature_ambient_mean"].mean().round(3)

start_datetime  start_datetime
2019            4                 284.961
                5                 286.852
                6                 298.090
                7                 297.648
                8                 294.405
                9                 290.175
                10                287.233
                11                280.998
                12                279.503
2020            1                 279.376
                2                 280.164
                3                 282.249
                4                 288.342
                5                 289.708
                7                 297.735
                8                 296.544
                9                 295.042
                10                285.253
                11                281.976
                12                278.645
2021            1                 276.693
                2                 282.870
                3                 283.015
   

### 15. For each bus compute the total time when the halt brake is active and the total time when the park brake is active. Compute also the ratio between those two times.

la status dei freni è registrato come una variabile booleana e le records avvengono ogni secondo

In [29]:
def to_date(x):
    return datetime.timedelta(seconds=x)

In [30]:
total_haltBrake = details.groupby("bus_number", as_index=False)["status_haltBrakeIsActive"].sum()
total_haltBrake["status_haltBrakeIsActive"] = total_haltBrake["status_haltBrakeIsActive"].apply(to_date)
total_haltBrake

,bus_number,status_haltBrakeIsActive
0,183,90 days 17:05:32
1,208,68 days 18:35:39


In [31]:
total_parkBrake = details.groupby("bus_number", as_index=False)["status_parkBrakeIsActive"].sum()
total_parkBrake["status_parkBrakeIsActive"] = total_parkBrake["status_parkBrakeIsActive"].apply(to_date)
total_parkBrake

,bus_number,status_parkBrakeIsActive
0,183,36 days 20:37:19
1,208,21 days 03:57:56


In [32]:
brakes = pd.merge(total_haltBrake, total_parkBrake, on = "bus_number")

In [33]:
brakes["ratio"]= round(brakes["status_haltBrakeIsActive"] / brakes["status_parkBrakeIsActive"], 3)
brakes

,bus_number,status_haltBrakeIsActive,status_parkBrakeIsActive,ratio
0,183,90 days 17:05:32,36 days 20:37:19,2.461
1,208,68 days 18:35:39,21 days 03:57:56,3.249


### 16. For each pair of stops that are consecutive in at least a trip, compute the average speed achieved when going from the first to the second stop.

In [34]:
#funzione che prende due liste e ne accoppia i valori nella stessa posizione
def merge_lists(list1, list2): 
    merged_list = [[list1[i], list2[i]] for i in range(0, len(list1))]     
    return merged_list

In [35]:
#Funzione che prende il dataset e restituisce le coppie di fermate consecutive come lista di stringhe

def consecutive_stops(df):
    stops = df[~(df["itcs_stopName"].isnull())]["itcs_stopName"].tolist()
    stops_1=stops[:-1]
    stops_2=stops[1:]
    return [stops_1[i] +" --> " +stops_2[i] for i in range(0, len(stops_1))]
    

In [36]:
#funzione che prende il dataset e restituisce una lista delle velocità medie tra ogni coppia di fermate del dataset
def average_speeds(df):
    avg_speeds=[]
    list_couples=[]
    indexes = df[~(df["itcs_stopName"].isnull())]["itcs_stopName"].index.tolist()
    list_couples=merge_lists(indexes[:-1], indexes[1:])
    for couple in list_couples:
        speeds=[]
        for i in range(couple[0],couple[1]):
            speeds.append(df.loc[i,"odometry_vehicleSpeed"])
            
        avg_speeds.append(round(np.mean(speeds), 3))
        
    return avg_speeds

In [37]:
list_of_df=[]

for trip in list_of_trips:
    
    list_of_cons_stops=[]
    list_of_cons_stops = consecutive_stops(trip)
    
    list_of_avg_speeds=[]
    list_of_avg_speeds = average_speeds(trip)
    
    dict_prov={}
    dict_prov={"consecutive_stops": list_of_cons_stops, "average_speed":list_of_avg_speeds}
    
    list_of_df.append(pd.DataFrame(dict_prov))
    

df_stops=pd.DataFrame()
df_stops=pd.concat(list_of_df, ignore_index=True)

In [38]:
df_stops.groupby("consecutive_stops").mean().round(3)

,average_speed
consecutive_stops,
"Zürich, Albisrank --> Zürich, Flurstrasse",4.428
"Zürich, Albisrank --> Zürich, Hubertus",4.388
"Zürich, Albisriederplatz --> Zürich, Altes Krematorium",3.614
"Zürich, Albisriederplatz --> Zürich, Friedhof Sihlfeld",4.729
"Zürich, Albisriederplatz --> Zürich, Hardplatz",4.491
...,...
"Zürich, Zwinglihaus --> Zürich, Goldbrunnenplatz",3.920
"Zürich, Zwinglihaus --> Zürich, Schmiede Wiedikon",3.271
"Zürich, Zwinglihaus --> Zürich,Kalkbreite/Bhf.Wiedikon",4.045
